In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    JavascriptException,
)
import pandas as pd
import time

In [18]:
districts = [
    "jakarta",
    "jakarta-barat",
    "jakarta-pusat",
    "jakarta-selatan",
    "jakarta-timur",
    "jakarta-utara",
]
genders = [
    "Semua",
    "Khusus Putra",
    "Khusus Putri",
    "Putra dan Campur",
    "Putri dan Campur",
]
price_starts = range(0, 15000000, 100000)

In [19]:
var = [
    "_id",
    "price_monthly",
    "latitude",
    "longitude",
    "gender",
    "area_city_keyword",
    "area_subdistrict",
    "status",
    "size",
    "fac_room",
    "fac_share",
    "fac_bath",
    "fac_near",
    "fac_park",
    "kos_rule",
    "fac_price",
    "owner_user_id",
    "building_year",
    "is_singgahsini",
    "is_apik",
    "is_elite",
    "number_success_owner_trx",
    "number_success_kos_trx",
]

In [ ]:
# Set up Selenium webdriver
chrome_options = Options()
chrome_options.add_argument("--window-size=1024,768")
driver = webdriver.Chrome(options=chrome_options)
url = "https://mamikos.com/booking/"

results = []

try:
    for district in districts:
        driver.get(url + district)
        for gender in genders:
            for price_start in price_starts:
                try:
                    # Wait until the filter appears
                    WebDriverWait(driver, 2).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, "div[data-v-424a8076]")
                        )
                    )

                    # Fill the gender type dropdown filter
                    filterType = driver.find_element(By.ID, "filterType")
                    filterType.send_keys(gender)

                    # Fill the price filter
                    filterPriceMin = driver.find_element(By.ID, "filterPriceMin")
                    filterPriceMin.send_keys(Keys.CONTROL + "a")
                    filterPriceMin.send_keys(price_start)
                    filterPriceMax = driver.find_element(By.ID, "filterPriceMax")
                    filterPriceMax.send_keys(Keys.CONTROL + "a")
                    filterPriceMax.send_keys(price_start + 100000)

                    # Click the submit button
                    submit = driver.find_element(
                        By.CSS_SELECTOR, 'button[type="submit"]'
                    )
                    submit.click()

                    while True:
                        # Wait until the listing cards appear
                        WebDriverWait(driver, 2).until(
                            EC.presence_of_element_located(
                                (By.CSS_SELECTOR, "div[data-v-c6752eba]")
                            )
                        )

                        # Collect all divs with class 'kost-rc__inner'
                        divs = driver.find_elements(
                            By.CSS_SELECTOR, "div.kost-rc__inner"
                        )

                        too_many_requests = False
                        # Click on the first div, then parse the content of the newly opened page
                        for div in divs:
                            time.sleep(0.1)
                            div.click()

                        for div in divs:
                            # Switch to the listing detail page
                            driver.switch_to.window(driver.window_handles[1])

                            # Execute the query to get the detail
                            detail = {}
                            while not detail:
                                try:
                                    if too_many_requests:
                                        driver.refresh()
                                    detail = driver.execute_script("return detail")
                                except JavascriptException:
                                    too_many_requests = True
                                    time.sleep(30)
                                    continue
                            detail = {key: detail[key] for key in var if key in detail}
                            detail["url"] = driver.current_url
                            results.append(detail)

                            # Close the page
                            driver.close()

                        # Switch back to the listing page
                        driver.switch_to.window(driver.window_handles[0])

                        # Click the next page button if it exists
                        next_page = driver.find_element(
                            By.XPATH, '//a[text()=">"][@tabindex="0"]'
                        )
                        next_page.click()
                except (TimeoutException, NoSuchElementException):
                    continue
finally:
    driver.quit()
    result = pd.DataFrame(results)

In [25]:
mamikos = pd.DataFrame(results)
len(mamikos)

160

In [ ]:
mamikos

In [22]:
pd.DataFrame(results).to_csv("mamikos_sample.csv", index="_id")